# Class Practice Session

## Goal:
- Use of texture coordinates as an additional vertex attribute.
- Texture mapping

## Change from Assignment 4
- Load image data
- Create a texture object and load texture data into GPU 
- Create a sampler from a texture object by specifying filter and wrap options
- Create a quad shaped clock-face geometry with texture coordinates
- vertex shader, fragment shader program for the clock face, and create a face render program using them.
- Create a face renderable
- before making the face render call
-- attach the texture sampler object to a texture unit
-- Set the sampler to texture unit number.
- Do the face render call

## Intialize

In [45]:
import pygame
import moderngl
import numpy
from math import cos, sin, pi, radians
import glm

pygame.init() # Initlizes its different modules. Display module is one of them.

width = 840
height = 480
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Class Practice: Instructor")
gl = moderngl.get_context() # Get Previously created context.
gl.info["GL_VERSION"]

'4.6.0 - Build 32.0.101.6881'

### Get the geometry data load it to a GPU buffer

In [46]:
diamond_position_data = [
    0.0, 0.8,
    -0.8, 0,
    0.8, 0,
    0.8, 0,
    -0.8, 0,
    0, -0.8
]
diamond_geometry = numpy.array(diamond_position_data).astype("float32")
#Push the Geometry Data to the GPU buffer.
diamond_geometry_buffer = gl.buffer(diamond_geometry) # Create a Vertex buffer object from the data

## Create shader programs

#### Vertex Shaders
The Vertex shader must set the value of a predefined vec4 variable gl_Position.

In [47]:
diamond_vertex_shader_code = '''
#version 330 core

// add attributes variables here
layout (location=0) in vec2 position;

uniform mat2 M;
uniform float scale;
uniform float radius;

void main() {
    vec2 displacement = radius * M * vec2(0,1);
    vec2 P = M * (scale * position) + displacement;
    gl_Position = vec4(P,0.0, 1.0);
}
'''
line_vertex_shader_code = '''
#version 330 core

// add attributes variables here
uniform mat2 M;
uniform float radius;

void main() {
    vec2 P = vec2(0);
    if (gl_VertexID > 0){
        P = radius * M * vec2(0,1);
    }
    gl_Position = vec4(P,0.0, 1.0);
}
'''

face_vertex_shader_code = '''
#version 330 core

out vec2 f_uv;
uniform mat2 M; 

void main() {
    
    vec2 P, uv;
    switch (gl_VertexID) {
        case 0:
        case 3:
            uv = vec2(0,0);
            P = vec2(-1,-1);
            break;
        case 1:
            uv = vec2(1,0);
            P = vec2(1,-1);
            break; 
        case 2:
        case 4: 
            uv = vec2(1,1);
            P = vec2(1,1);
            break;
        case 5:
            uv = vec2(0,1);
            P = vec2(-1,1);
            break;
    }
    gl_Position = vec4(P*M,0.0, 1.0);
}
'''

#### Fragment Shaders
For rendering fragment shader must output something (minimally a vec4 value) to the framebuffer 

In [48]:
fragment_shader_code = '''
#version 330 core

out vec4 color;
uniform vec3 inColor;

void main() {
   
    color = vec4(inColor,1);
}
'''

face_fragment_shader_code = '''
#version 330 core

out vec4 color;

in vec2 f_uv; 
uniform sampler2D map; 

void main() {

    color = texture(map, f_uv);
}
'''

### Create Shader program from the Vertex and Fragment shader code

In [49]:
diamond_program = gl.program(
    vertex_shader= diamond_vertex_shader_code,
    fragment_shader=fragment_shader_code
)

line_program = gl.program(
    vertex_shader= line_vertex_shader_code,
    fragment_shader= fragment_shader_code
)
face_program = gl.program(
    vertex_shader= face_vertex_shader_code,
    fragment_shader= face_fragment_shader_code
)

#### Create connection between the data in buffer object and vertex attribute in vertex program 
Connection is created by function ctx.vertex_array(program: Program, content: list).  
- $list# is a python list of (buffer, format, attribute name(s))

See https://moderngl.readthedocs.io/en/latest/reference/vertex_array.html 

In [50]:
diamond_renderable = gl.vertex_array(diamond_program,
    [( diamond_geometry_buffer, "2f", "position" ) ]
)

line_renderable = gl.vertex_array(line_program,
    []
)
face_renderable = gl.vertex_array(face_program, [])

##  Create Texture object and specify filter and wrap options.
__Load image and get the data in a sequence of bytes__  
pygame.image.load() will return a Suface object that has image data.  
texture_img = pygame.image.load(image_file) 
texture_data = pygame.image.tobytes(texture_img,channels,True) \#tobytes(Surface, format=\<"RGB" or "RGBA"\>, \<to Flip or no to flip True/False\>) -> bytes    
\#w, h = texture_img.get_size()   

__Create Texture object and send the data__  
ctx.texture(size = \<size tuple\>, data=\<texture data in bytes\>,components=\<Number of color channels>) \# returns texture object filled with data

__Create a sampler object that associates filter and wrap options with the texture__  
 ctx.sampler(texture=\<texture object\>, filter=(\<Minification filter\>, \<magnification filter\>), repeat_x=\<True/False\>, repeat_y=\<True/False\>) \# returns a sampler object

The sampler object must be attached to a texture unit before it can be used in the shader program.  
\<sampler object\>.use(n)

In [51]:
texture_img = pygame.image.load("clockFace.png").convert_alpha()
texture_data = pygame.image.tobytes(texture_img, "RGB", True)
faceTexture = gl.texture(texture_img.get_size(), data = texture_data, components=3)
faceSampler = gl.sampler(texture=faceTexture, filter=(gl.LINEAR, gl.LINEAR))

In [52]:
texture_img.get_size()

(1660, 1655)

## Render loop

In [53]:
running = True
clock = pygame.time.Clock()
alpha = 0

while running:   
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    # event.key == 27 means Escape key is pressed.
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
    aspect = width/height
    # create the aspect ratio correction matrix
    
    M_aspect = glm.mat2(1/aspect, 0, 0, 1) if aspect > 1 else glm.mat2(1, 0, 0, aspect)

    #Create the rotation matrix
    # ... Part of assignment 4
    # M = M_aspect * M_rotate
    M = M_aspect
    
    ### Add render code below
    # Clear the display window with the specified R, G, B values using function ctx.clear(R, G, B)
    gl.clear(0.5,0.5,0.0)

    faceSampler.use(0)
    face_program["map"] = 0
    face_program["M"].write(M)
    face_renderable.render(moderngl.TRIANGLES, vertices=6)
    
    # Make one or more Render Calls to instruct the GPU to render by executing the shader program with the provided data.

    diamond_program["M"].write(M)
    
    diamond_program["scale"] = 0.1
    diamond_program["inColor"] = 1, 1, 0
    
    diamond_program["radius"] = 0.8
    diamond_renderable.render()
    
    diamond_program["radius"] = 0.0
    diamond_renderable.render()
    
    line_program["M"].write(M)
    line_program["radius"] = 0.8
    line_program["inColor"] = 0, 0, 0
    line_renderable.render(moderngl.LINES, vertices=2)  
    
    pygame.display.flip()
    clock.tick(60)  # limits FPS to 10
    alpha = alpha + 1
    if alpha > 360:
        alpha = 0

pygame.display.quit()

In [54]:
print(M_aspect)

[     0.571429 ][            0 ]
[            0 ][            1 ]


In [55]:
print(M)

[     0.571429 ][            0 ]
[            0 ][            1 ]
